In [2]:
from ngsolve import *
from ngsolve.webgui import Draw
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *
import time # Added for visual delay

In [3]:
from netgen.occ import *
shape = Rectangle(1,0.1).Face()
shape.edges.Max(X).name="right"
shape.edges.Min(X).name="left"
shape.edges.Max(Y).name="top"
shape.edges.Min(Y).name="bot"
shape.vertices.Min(X+Y).maxh=0.01
shape.vertices.Min(X-Y).maxh=0.01
mesh = Mesh(OCCGeometry(shape, dim=2).GenerateMesh(maxh=0.05))
Draw (mesh);E, nu = 210, 0.2
mu  = E / 2 / (1+nu)
lam = E * nu / ((1+nu)*(1-2*nu))

def C(u):
    F = Id(2) + Grad(u)
    return F.trans * F

def NeoHooke (C):
    return 0.5*mu*(Trace(C-Id(2)) + 2*mu/lam*Det(C)**(-lam/2/mu)-1)

tau = 0.02
tend = 20
force = CF( (0, -1) )

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

In [4]:
E, nu = 210, 0.2
mu  = E / 2 / (1+nu)
lam = E * nu / ((1+nu)*(1-2*nu))

def C(u):
    F = Id(2) + Grad(u)
    return F.trans * F

def NeoHooke (C):
    return 0.5*mu*(Trace(C-Id(2)) + 2*mu/lam*Det(C)**(-lam/2/mu)-1)

tau = 0.02
tend = 20
force = CF( (0, -1) )

In [5]:
fes = VectorH1(mesh, order=3, dirichlet="left")
u,v = fes.TnT()

gfu = GridFunction(fes)
gfv = GridFunction(fes)
gfa = GridFunction(fes)
gfuold = GridFunction(fes)
gfvold = GridFunction(fes)
gfaold = GridFunction(fes)

bfa = BilinearForm(fes)
bfa += Variation(NeoHooke(C(u))*dx)

vel_new = 2/tau * (u-gfuold) - gfvold
acc_new = 2/tau * (vel_new-gfvold) - gfaold

bfa += acc_new*v*dx
bfa += -force*v*dx

In [6]:
def MyNewton(a, u, printing=False):
    for it in range(10):
        res = a.Apply(gfu.vec)
        if printing:
            print ("it=", it, "err=", Norm(res))
        a.AssembleLinearization(gfu.vec)
        inv = a.mat.Inverse(fes.FreeDofs(), inverse="sparsecholesky") 
        gfu.vec.data -= inv*res

In [7]:
from ngsolve.solvers import Newton
sceneu = Draw (gfu, deformation=True)
scenev = Draw (gfv)
scenea = Draw (gfa)
gfu.vec[:] = 0
t = 0
while t < tend:
    t += tau
    MyNewton(a=bfa, u=gfu, printing=False)
    # Newton(a=bfa, u=gfu, printing=False, inverse="sparsecholesky")

    gfv.vec[:] = 2/tau * (gfu.vec-gfuold.vec) - gfvold.vec
    gfa.vec[:] = 2/tau * (gfv.vec-gfvold.vec) - gfaold.vec

    sceneu.Redraw()
    scenev.Redraw()
    scenea.Redraw()
    
    gfuold.vec[:] = gfu.vec
    gfvold.vec[:] = gfv.vec
    gfaold.vec[:] = gfa.vec

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

KeyboardInterrupt: 

## Dynamics of beams

start from the notebook dynamics, and replace the Dirichlet boundary condition by mean value constraints for the displacement (but not for the rotation).

In [8]:
fes_u = VectorH1(mesh, order=3)
fes_lm = NumberSpace(mesh)
fes = fes_u * fes_lm * fes_lm

(u, lam_x, lam_y), (v, mu_x, mu_y) = fes.TnT()
sol = GridFunction(fes, name="solution")
gfu = sol.components[0]      # Displacement part of the solution
gflam_x = sol.components[1]  # LM for x-constraint
gflam_y = sol.components[2]  # LM for y-constraint


gfu_old = GridFunction(fes_u, name="u_old") # u at start of step
gfv_old = GridFunction(fes_u, name="v_old") # v at start of step
gfa_old = GridFunction(fes_u, name="a_old") # a at start of step

gfv_cur = GridFunction(fes_u, name="v_current") # v calculated at end of step
gfa_cur = GridFunction(fes_u, name="a_current") # a calculated at end of step

In [9]:
bfa = BilinearForm(fes, symmetric=False) 
bfa += Variation(NeoHooke(C(u)).Compile()*dx)

vel_new = 2/tau * (u - gfu_old) - gfv_old
acc_new = 2/tau * (vel_new - gfv_old) - gfa_old
bfa += (acc_new * v * dx).Compile()

bfa += (-force * v * dx).Compile()

bnd_left = mesh.Boundaries("left")
bfa += (lam_x * v[0] * ds(definedon=bnd_left)).Compile()
bfa += (mu_x * u[0] * ds(definedon=bnd_left)).Compile()
bfa += (lam_y * v[1] * ds(definedon=bnd_left)).Compile()
bfa += (mu_y * u[1] * ds(definedon=bnd_left)).Compile()
left_boundary_integration = ds(definedon=bnd_left)

In [12]:
def MyNewton(a, u_full_gf, printing=False, max_iter=25, tolerance=1e-5):
    """
    Newton solver adapted for the combined system.
    'a': The BilinearForm defined on the combined space (e.g., bfa).
    'u_full_gf': The GridFunction on the combined space holding the full solution (e.g., sol).
    """

    full_vec = u_full_gf.vec
    initial_norm_res = -1.0

    for it in range(max_iter):
        res = a.Apply(full_vec)
        norm_res = Norm(res)

        if it == 0: initial_norm_res = norm_res
        if printing:
            print (f"Newton it={it}, ||res||={norm_res:.3e}")

        if norm_res < tolerance:
            return True

        if it > 4 and norm_res > initial_norm_res * 100:
             print(f"Newton residual increasing significantly (||res||={norm_res:.3e}). Stopping.")
             return False

        a.AssembleLinearization(full_vec)

        try:
            inv = a.mat.Inverse(a.space.FreeDofs(), inverse="pardiso")
            full_vec.data -= inv * res
        except Exception as e:
            print(f"Error during matrix inversion/solve: {e}")
            print("Newton iteration failed.")
            return False # Failed

    print(f"Newton did NOT converge after {max_iter} iterations! ||res||={norm_res:.3e}")
    return False

In [14]:
sceneu = Draw (gfu, deformation=True, min=0, max=1.0, autoscale=False) 
sol.vec[:] = 0      # Initialize full solution vector
gfu_old.vec[:] = 0  # Initialize previous displacement
gfv_old.vec[:] = 0  # Initialize previous velocity
gfa_old.vec[:] = 0  # Initialize previous acceleration

t = 0
cnt = 0
output_every = max(1, int(0.05 / tau))
start_time = time.time()

success = True
while t < tend and success:
    t += tau
    gfu_old.vec.data = gfu.vec # gfu is sol.components[0]
    success = MyNewton(a=bfa, u_full_gf=sol, printing=False)

    if not success:
        print(f"Solver failed at t={t:.4f}. Stopping simulation.")
        break
    gfv_cur.vec.data = 2/tau * (gfu.vec - gfu_old.vec) - gfv_old.vec
    gfa_cur.vec.data = 2/tau * (gfv_cur.vec - gfv_old.vec) - gfa_old.vec
    gfv_old.vec.data = gfv_cur.vec
    gfa_old.vec.data = gfa_cur.vec
    cnt += 1
    if cnt % output_every == 0:
        sceneu.Redraw()
        print(f"t = {t:.3f} / {tend:.3f} ", end='\r')

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

t = 20.000 / 20.000 

In [16]:
try:
    left_bnd_region = mesh.Boundaries("left")
    avg_u_left_integral = Integrate(gfu, mesh, definedon=left_bnd_region) # This is a VectorD result
    left_edge_length = Integrate(CF(1), mesh, definedon=left_bnd_region) # This is a float result

    print(f"Integral of u over left edge: ({avg_u_left_integral[0]:.3e}, {avg_u_left_integral[1]:.3e})")
    print(f"Length of left edge: {left_edge_length:.4f}")
    if left_edge_length > 1e-10:
        avg_u_x = avg_u_left_integral[0] / left_edge_length
        avg_u_y = avg_u_left_integral[1] / left_edge_length
        print(f"Final average displacement on left edge: ({avg_u_x:.3e}, {avg_u_y:.3e})")
    else:
        print("Left edge length is too small to calculate average displacement.")

except Exception as e:
    print(f"An error occurred during final boundary integration check: {e}")

Integral of u over left edge: (2.168e-19, 0.000e+00)
Length of left edge: 0.1000
Final average displacement on left edge: (2.168e-18, 0.000e+00)


simpulate a double pendulum by coupling two beams via mean value constraints

In [33]:
width  = 0.1
depth  = 0.1
length = 1.0

beam1 = OrthoBrick(Pnt(-width/2, 0.0, -depth/2), Pnt(width/2, length, depth/2))
beam2 = OrthoBrick(Pnt(-width/2, -length, -depth/2), Pnt(width/2, 0.0, depth/2))

geo = CSGeometry()
geo.Add(beam1)
geo.Add(beam2)
mesh = Mesh(geo.GenerateMesh(maxh=0.1))


In [34]:
fes_u1 = VectorH1(mesh, order=3) # Displacement space for beam 1
fes_u2 = VectorH1(mesh, order=3) # Displacement space for beam 2
fes_lm  = NumberSpace(mesh) # Lagrange multiplier space for the coupling constraint

fes = fes_u1 * fes_u2 * fes_lm * fes_lm
(u1, u2, lam_x, lam_y), (v1, v2, mu_x, mu_y) = fes.TnT()

sol = GridFunction(fes, name="solution")

gfu1 = sol.components[0]  # Displacement for beam 1
gfu2 = sol.components[1]  # Displacement for beam 2
gfu1_old = GridFunction(fes_u1, name="u1_old")
gfv1_old = GridFunction(fes_u1, name="v1_old")
gfa1_old = GridFunction(fes_u1, name="a1_old")
gfu2_old = GridFunction(fes_u2, name="u2_old")
gfv2_old = GridFunction(fes_u2, name="v2_old")
gfa2_old = GridFunction(fes_u2, name="a2_old")

gfv1_cur = GridFunction(fes_u1, name="v1_current")
gfa1_cur = GridFunction(fes_u1, name="a1_current")
gfv2_cur = GridFunction(fes_u2, name="v2_current")
gfa2_cur = GridFunction(fes_u2, name="a2_current")

bfa = BilinearForm(fes, symmetric=False)

In [46]:
def NeoHooke (C):
    return 0.5*mu*(Trace(C-Id(3)) + 2*mu/lam*Det(C)**(-lam/2/mu)-1)

def C(u):
    F = Id(3) + Grad(u)
    return F.trans * F

In [53]:
bfa += Variation(NeoHooke(C(u1)).Compile() * dx(definedon=mesh.Materials([0])))
bfa += Variation(NeoHooke(C(u2)).Compile() * dx(definedon=mesh.Materials([1])))
tau = 0.01      # time step size
tend = 2.0      # end time
vel1_new = 2/tau * (u1 - gfu1_old) - gfv1_old
acc1_new = 2/tau * (vel1_new - gfv1_old) - gfa1_old
bfa += (acc1_new * v1 * dx(1)).Compile()

TypeError: __call__(): incompatible function arguments. The following argument types are supported:
    1. (self: ngsolve.comp.DifferentialSymbol, definedon: Optional[Union[ngsolve.comp.Region, str]] = None, element_boundary: bool = False, element_vb: ngsolve.comp.VorB = <VorB.VOL: 0>, skeleton: bool = False, bonus_intorder: int = 0, intrules: dict[ngsolve.fem.ET, ngsolve.fem.IntegrationRule] = {}, deformation: ngsolve.comp.GridFunction = None, definedonelements: pyngcore.pyngcore.BitArray = None) -> ngsolve.comp.DifferentialSymbol

Invoked with: <ngsolve.comp.DifferentialSymbol object at 0x7fa1c3fb75b0>, 1